<a href="https://colab.research.google.com/github/mmc509/my_deep_learning_experience/blob/main/FAQBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow_hub
!pip install tensorflow_text

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import pandas as pd
import csv
import numpy as np

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 66.4 MB/s eta 0:00:00


In [2]:
df = pd.read_csv("training_data.csv")
df.head(5)

,query,label
0,can you explain how i can adjust ambient light...,0
1,can you explain how i can adjust brightness of...,0
2,can you explain how i can adjust brightness of...,0
3,can you explain how i can adjust instrument li...,0
4,can you explain how i can adjust brightness of...,0


In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['query'],df['label'], stratify=df['label'])
X_train.head(4)

4570      how can i adjust your brightness of the instru...
20249            where can you adjust instrument brightness
2960      could you please tell me how to adjust our bri...
131082    where I can get an brightness of the instrumen...
Name: query, dtype: object

In [4]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

array([[0.9911088]], dtype=float32)

In [5]:
# Bert layers: for each sentence, there's 128 words, for each word, there's 768 features(dimentions).
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(5, activation='softmax', name="output")(l)

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

model.summary()

METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy')
]

model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=METRICS)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_word_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128),                                                          
                                 'input_type_ids':                                                
                                (None, 128)}                                                  

In [6]:
model.fit(X_train, y_train, epochs=10)

Epoch 1/10
3211/3211 [==============================] - 1078s 332ms/step - loss: 0.5049 - accuracy: 0.2118
Epoch 2/10
3211/3211 [==============================] - 1072s 334ms/step - loss: 0.2182 - accuracy: 0.2120
Epoch 3/10
3211/3211 [==============================] - 1066s 332ms/step - loss: 0.1652 - accuracy: 0.2122
Epoch 4/10
3211/3211 [==============================] - 1065s 332ms/step - loss: 0.1443 - accuracy: 0.2122
Epoch 5/10
3211/3211 [==============================] - 1065s 332ms/step - loss: 0.1331 - accuracy: 0.2122
Epoch 6/10
3211/3211 [==============================] - 1064s 331ms/step - loss: 0.1233 - accuracy: 0.2122
Epoch 7/10
3211/3211 [==============================] - 1065s 332ms/step - loss: 0.1176 - accuracy: 0.2122
Epoch 8/10
3211/3211 [==============================] - 1064s 331ms/step - loss: 0.1145 - accuracy: 0.2123
Epoch 9/10
3211/3211 [==============================] - 1065s 332ms/step - loss: 0.1094 - accuracy: 0.2123
Epoch 10/10
3211/3211 [==============

In [7]:
model.evaluate(X_test, y_test)

1071/1071 [==============================] - 359s 335ms/step - loss: 0.0564 - accuracy: 0.2123


[0.056408949196338654, 0.21232405304908752]

In [8]:
y_predicted = model.predict(X_test)

1071/1071 [==============================] - 355s 331ms/step


In [9]:
# y_predicted.shape

(34242, 5)

In [14]:
model.save("./my_trained_model")

In [19]:
# zip -r my_model.zip ./my_trained_model
import shutil
shutil.make_archive("trained_model", 'zip', "./my_trained_model")

'/content/trained_model.zip'

In [26]:
new_model = tf.keras.models.load_model('my_trained_model')

In [27]:
questions = [
    'could you tell me how to check battery',#1
    'could you tell me how to adjust the brightness',#0
    'how can i locate the battery',#3
    'can you help me to find out where the car battery is',#3
    "do you know the location of my battery"#3
]
results = new_model.predict(questions)
for r in results:
  print(np.where(r == np.amax(r)))

1/1 [==============================] - 1s 864ms/step
(array([1]),)
(array([0]),)
(array([3]),)
(array([3]),)
(array([3]),)
